In [ ]:
"""\
# Copyright (C) 2024 Jesús Bautista Villar <jesbauti20@gmail.com>
"""
!python3 --version

In [ ]:
import numpy as np
import os

# Animation tools
from IPython.display import HTML
from matplotlib.animation import PillowWriter

# Import the Swarm Systems Lab Simulator
from ssl_simulator import SimulationEngine, load_data, createDir
from ssl_simulator.robot_models import Unicycle2D
from ssl_simulator.data_manager import load_sim
from ssl_simulator.visualization import set_paper_parameters
set_paper_parameters(fontsize=15)

# Import custom functions and classes
from visualization import PlotBasicGvf
from visualization import AnimationXY, AnimationXY, AnimationXYPhi
from controllers import GvfIK
from ssl_simulator.gvf_trajectories import GvfEllipse

# -------------------------------------------------------------------------------------

# Data and output directories
DATA_DIR = os.path.join("..", "data")
OUTPUT_DIR = os.path.join("..", "output")
SIMDATA_FILE = os.path.join(OUTPUT_DIR, "data.csv")

createDir(OUTPUT_DIR)

## ICRA video animations

### Animation of the simulation

In [ ]:
# Define the initial state
N = 1

p = np.ones((N,2)) * np.array([[150,-200]])
speed = np.ones((N)) * np.array([14])
theta = np.ones((N)) * np.array([-90])

x0 = [p, speed, theta]

# Controller settings
A = np.ones((N)) * 0.3
omega = np.ones((N)) * np.pi/4

# --------------------------------
# Generate the trajectory to be followed
a, b = 100, 70
XYoff, alpha = [0, 0], 0

gvf_traj = GvfEllipse(XYoff,alpha,a,b)

# Select the robot model
robot_model = Unicycle2D(x0)

# Initialize the controller
s, ke, kn = 1, 0.35, 1
controller = GvfIK(gvf_traj, s, ke, kn, A, omega)


# Then, construct the Simulator Engine
dt = 0.01
simulator = SimulationEngine(robot_model, controller, time_step=dt, log_filename=SIMDATA_FILE)

# --------------------------------
# Run the simulation and save data
tf = 10

simulator.run(tf)
# --------------------------------

In [ ]:
# Load previously saved simulation data from the specified file
simulation_data = load_sim(SIMDATA_FILE, debug=False)

# Initialize the animator with the loaded simulation data
animator = AnimationXY(
    gvf_traj, simulation_data, 
    dpi=100, figsize=(6,6), xlims=[-200,200], ylims=[-250,150])

anim = animator.gen_animation(fps=1/dt)
HTML(anim.to_html5_video()) # It takes a loooot of time...